In [ ]:

!pip install google-cloud-vision sympy opencv-python-headless matplotlib gradio numpy


import cv2
import numpy as np
import sympy as sp
import json
import re
import gradio as gr
from google.cloud import vision
from PIL import Image


API_JSON_PATH = "/content/handwriiten-8d368abbd4ac.json"
with open(API_JSON_PATH, "r") as f:
    credentials = json.load(f)
client = vision.ImageAnnotatorClient.from_service_account_info(credentials)


def process_equation(image):
    if image is None:
        return "⚠️ Please upload an image!", None, None, None, None

    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)


    _, encoded_image = cv2.imencode(".jpg", image)
    content = encoded_image.tobytes()
    vision_image = vision.Image(content=content)
    response = client.text_detection(image=vision_image)
    texts = response.text_annotations


    equation_text = None
    if texts:
        raw_text = texts[0].description.strip().replace("\n", " ")
        equation_text = re.sub(r"[^0-9a-zA-Z=+\-*/^().]", "", raw_text)
        equation_text = equation_text.replace("^", "**")
        equation_text = re.sub(r"([0-9])([a-zA-Z])", r"\1*\2", equation_text)
        equation_text = equation_text.replace("AdobeStock", "").strip()
    else:
        return "⚠️ No equation detected! Please upload a clearer image.", None, None, None, None


    solution = None
    if "=" in equation_text:
        try:
            variables = list(set(re.findall(r"[a-zA-Z]", equation_text)))
            symbols = {var: sp.Symbol(var) for var in variables}
            expr = sp.sympify(equation_text.replace("=", "-(") + ")", locals=symbols)
            solution = sp.solve(expr, list(symbols.values())[0])
        except Exception as e:
            solution = f"❌ Error solving equation: {e}"


    original_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    processed_pil = Image.fromarray(thresh)

    return equation_text, solution if solution else "⚠️ Not solvable / expression only", original_pil, processed_pil


interface = gr.Interface(
    fn=process_equation,
    inputs=gr.Image(type="pil", label="📥 Upload a Handwritten Math Equation"),
    outputs=[
        gr.Textbox(label="✏️ Recognized Equation"),
        gr.Textbox(label="🧮 Solution (if solvable)"),
        gr.Image(label="🖼️ Original Image (Uploaded)"),
        gr.Image(label="🔍 Processed Image (Enhanced for OCR)"),
    ],
    title="✍️ AI-Powered Handwritten Equation Solver",
    description="""
    **Upload a clear image of a handwritten mathematical equation**, and our AI will:
    - 📝 **Recognize and extract** the equation.
    - 🧮 **Solve it if possible** (for algebraic equations).
    - 🚀 **Works best with simple algebraic expressions**.
    - 📸 **Drag & Drop or Click to Upload** your equation image.
    """,
    allow_flagging="never",
    theme="huggingface",
)


interface.launch(share=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1102: UserWarning: Cannot load huggingface. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/huggingface (Request ID: Root=1-67dff5df-10f2aeb07315684476b986d0;3f777e45-cd3b-4651-b59b-821bc20a423c)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.11/dist-packages/gradio/interface.py:403: Use

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://580ba4cd2c7d574251.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
